In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import xgboost as xgb

In [2]:
# Parameters for tunning

BASE_IMG_FOLDER = './VC_2425_Project_public/images/'
# BASE_IMG_FOLDER = './all_images/'

# Parameters for the image processing
SHOW_DEBUG_IMGS = "NONE" # ["LINES", "WARP", "ALL", "NONE"]
CORNER_HORSE_TEMPLATE_PATH = "./cornerHorse_templates/"
SAVE_DEBUG_IMGS = False

In [3]:
# Load the images for the corner horses and store them in a list
CORNER_HORSES_TEMPLATES = []
for filename in os.listdir(CORNER_HORSE_TEMPLATE_PATH):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        img = cv2.imread(os.path.join(CORNER_HORSE_TEMPLATE_PATH, filename))
        CORNER_HORSES_TEMPLATES.append(img)
    else:
        continue

In [4]:
import pickle
REAL_MATRICES = {}
# Load the real matrix from pickle file
try:
    with open('real_matrices.pkl', 'rb') as f:
        REAL_MATRICES = pickle.load(f)
    print(f"Loaded real matrices for {len(REAL_MATRICES)} images")
except FileNotFoundError:
    print("Real matrices file not found!")
    REAL_MATRICES = {}

Loaded real matrices for 50 images


In [5]:
def show_original_and_gray(image_path):
    original_img = cv2.imread(image_path)
    rgb_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    
    # Wood removal by color thresholding (targeting brown/wooden colors)
    hsv_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2HSV)
    
    # Define range for brown/wooden colors in HSV
    lower_brown = np.array([2, 11, 11])
    upper_brown = np.array([60, 255, 255]) 
    
    # Create mask for wood
    wood_mask = cv2.inRange(hsv_img, lower_brown, upper_brown)
    
    # Invert the mask to keep non-wood parts
    wood_mask_inv = cv2.bitwise_not(wood_mask)
    
    # Apply the mask to the original image
    no_wood_img = cv2.bitwise_and(original_img, original_img, mask=wood_mask_inv)
    
    
    # Convert to grayscale
    gray_img = cv2.cvtColor(no_wood_img, cv2.COLOR_BGR2GRAY)
    

    # Display images to show the process
    if SHOW_DEBUG_IMGS == "ALL":
        plt.figure(figsize=(10, 5))

        # Original image - left
        plt.subplot(1, 2, 1)
        plt.imshow(rgb_img)
        plt.axis('off')
        plt.title('Original RGB Image')

        # Final grayscale image - right
        plt.subplot(1, 2, 2)
        plt.imshow(gray_img, cmap='gray')
        plt.axis('off')
        plt.title('Without Wood Grayscale Image')

        plt.tight_layout()
        plt.show()

    return original_img, gray_img

In [6]:
def preprocess_image(gray_img, blur_kernel_size=17, intensity_factor=1.3, laplacian_kernel_size=3):
    blurred = cv2.GaussianBlur(gray_img, (blur_kernel_size, blur_kernel_size), 0)
    adjusted_img = cv2.convertScaleAbs(blurred, alpha=intensity_factor, beta=0)

    # Morphological opening to remove small details like pieces
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    opened = cv2.morphologyEx(adjusted_img, cv2.MORPH_OPEN, kernel)

    laplacian = cv2.Laplacian(opened, cv2.CV_64F, ksize=laplacian_kernel_size)
    laplacian = cv2.convertScaleAbs(laplacian)

    # OTSU + optional manual offset to suppress weak edges
    otsu_thresh_val, _ = cv2.threshold(laplacian, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    _, binary_img = cv2.threshold(laplacian, otsu_thresh_val + 20, 255, cv2.THRESH_BINARY)

    return binary_img

In [7]:
def detect_lines(binary_img, min_line_length=50, max_line_gap=50):
    # Canny edge detection with lower threshold
    canny_image = cv2.Canny(binary_img, 50, 200)  # Tuning thresholds to capture better edges

    # Use dilation to reinforce edges
    kernel = np.ones((13, 13), np.uint8)
    dilation_image = cv2.dilate(canny_image, kernel, iterations=1)
    
    # Hough Lines transform for line detection
    lines = cv2.HoughLinesP(dilation_image, 1, np.pi / 180, threshold=500, 
                            minLineLength=min_line_length, maxLineGap=max_line_gap)

    # Create an image to store the detected lines
    black_image = np.zeros_like(dilation_image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(black_image, (x1, y1), (x2, y2), (255, 255, 255), 2)
    
    return black_image

In [8]:
def remove_noise_components(line_img, min_area=1000, keep_largest=True):
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(line_img, connectivity=8)
    
    # Create a black image to store the final result
    final_image = np.zeros_like(line_img)
    
    if keep_largest:
        # Get the areas of all components (ignoring the background)
        areas = stats[1:, cv2.CC_STAT_AREA]
        
        # Find the label of the largest component
        max_label = 1 + np.argmax(areas)
        
        # Keep only the largest component
        final_image[labels == max_label] = 255
    else:
        # If not keeping only the largest component, keep components above the min_area threshold
        for label in range(1, num_labels):
            area = stats[label, cv2.CC_STAT_AREA]
            if area >= min_area:
                final_image[labels == label] = 255

    if (SHOW_DEBUG_IMGS == "ALL" or SHOW_DEBUG_IMGS == "LINES"):
        plt.subplot(1, 3, 1)
        plt.title("Only Board Lines (Filtered)", fontsize=12)
        plt.imshow(final_image, cmap="gray")
        plt.axis('off')

    return final_image

In [9]:
def find_chessboard_contour(line_img):
    contours, _ = cv2.findContours(line_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    if contours:
        return contours[0]
    return None

In [10]:
def order_corners(corners):
    sorted_by_y = sorted(corners, key=lambda p: p[1])
    top_two = sorted(sorted_by_y[:2], key=lambda p: p[0])
    bottom_two = sorted(sorted_by_y[2:], key=lambda p: p[0])
    return np.array([top_two[0], top_two[1], bottom_two[0], bottom_two[1]], dtype="float32")

In [11]:
def warp_chessboard(gray_img, original_img, contour, board_size=800):
    epsilon = 0.05 * cv2.arcLength(contour, True)
    approx_corners = cv2.approxPolyDP(contour, epsilon, True)
    if len(approx_corners) == 4:
        corners = np.squeeze(approx_corners)
        ordered_corners = order_corners(corners)
        dst_corners = np.array([
            [0, 0], [board_size - 1, 0],
            [0, board_size - 1], [board_size - 1, board_size - 1]
        ], dtype="float32")
        matrix = cv2.getPerspectiveTransform(ordered_corners, dst_corners)
        warped_board = cv2.warpPerspective(gray_img, matrix, (board_size, board_size))
        
        # Also warp the original RGB image
        warped_original = cv2.warpPerspective(original_img, matrix, (board_size, board_size))
        warped_original_rgb = cv2.cvtColor(warped_original, cv2.COLOR_BGR2RGB)
        
        if (SHOW_DEBUG_IMGS == "ALL" or SHOW_DEBUG_IMGS == "LINES" or SHOW_DEBUG_IMGS == "WARP"):
            plt.subplot(1, 3, 2)
            plt.title("Warped Chessboard (Grayscale)", fontsize=12)
            plt.imshow(warped_board, cmap="gray")
            plt.axis('off')
            
            plt.subplot(1, 3, 3)
            plt.title("Warped Chessboard (Original)", fontsize=12)
            plt.imshow(warped_original_rgb)
            plt.axis('off')
            
        # Return the warp matrix as well to allow for reverting the transform
        return warped_board, warped_original_rgb, matrix, ordered_corners, dst_corners
    else:
        print("Error: Did not find exactly 4 corners!")
        return None, None, None, None, None

In [12]:
def process_chessboard_image(image_path):
    original_img, gray_img = show_original_and_gray(image_path)
    otsu_binary = preprocess_image(gray_img)
    line_img = detect_lines(otsu_binary)
    clean_line_img = remove_noise_components(line_img)
    contour = find_chessboard_contour(clean_line_img)
    if contour is not None:
        return warp_chessboard(gray_img, original_img, contour)
    else:
        print("Chessboard contour not found!")
        return None

In [13]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def detect_grid_lines(image_rgb, show_lines=True, edge_threshold=20):
    height, width = image_rgb.shape[:2]

    gray = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)
    blurred = cv2.GaussianBlur(gray, (17, 13), 0)

    edges = cv2.Canny(blurred, 50, 150)
    edges = cv2.dilate(edges, np.ones((3, 3), np.uint8), iterations=1)

    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=80, minLineLength=100, maxLineGap=10)
    
    if lines is None:
        print("No lines found")
        return [], []

    horizontal_lines = []
    vertical_lines = []

    for x1, y1, x2, y2 in lines[:, 0]:
        angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
        if abs(angle) < 10:
            horizontal_lines.append((x1, y1, x2, y2))
        elif abs(angle - 90) < 10 or abs(angle + 90) < 10:
            vertical_lines.append((x1, y1, x2, y2))

    def merge_line_coords(lines, axis='y', threshold=15):
        if not lines:
            return []
        coords = [int((y1 + y2) / 2) if axis == 'y' else int((x1 + x2) / 2) for x1, y1, x2, y2 in lines]
        coords = sorted(coords)
        merged = []
        current = coords[0]
        for val in coords[1:]:
            if abs(val - current) < threshold:
                current = int((current + val) / 2)
            else:
                merged.append(current)
                current = val
        merged.append(current)
        return merged

    merged_horizontal = merge_line_coords(horizontal_lines, axis='y')
    merged_vertical = merge_line_coords(vertical_lines, axis='x')

    # Remove lines too close to image edges
    merged_horizontal = [y for y in merged_horizontal if edge_threshold < y < (height - edge_threshold)]
    merged_vertical = [x for x in merged_vertical if edge_threshold < x < (width - edge_threshold)]

    if show_lines:
        merged_image = image_rgb.copy()
        for y in merged_horizontal:
            cv2.line(merged_image, (0, y), (merged_image.shape[1], y), (255, 0, 0), 3)
        for x in merged_vertical:
            cv2.line(merged_image, (x, 0), (x, merged_image.shape[0]), (255, 0, 0), 3)

        plt.figure(figsize=(8, 8))
        plt.imshow(merged_image)
        plt.title("Filtered Merged Grid Lines")
        plt.axis("off")
        plt.show()

    return merged_horizontal, merged_vertical

In [14]:
def find_horse_template_matching(image_rgb, image_name=None):
    img_rgb = image_rgb.copy()
    height, width = img_rgb.shape[:2]
    
    best_match_val = -np.inf
    best_match_loc = None
    best_template_shape = None
    best_template_index = None

    for idx, template_rgb in enumerate(CORNER_HORSES_TEMPLATES):
        if template_rgb is None:
            print(f"Template at index {idx} is None.")
            continue

        h, w = template_rgb.shape[:2]
        
        # Template matching
        res = cv2.matchTemplate(img_rgb, template_rgb, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        
        if max_val > best_match_val:
            best_match_val = max_val
            best_match_loc = max_loc
            best_template_shape = (h, w)
            best_template_index = idx

    print(f"Best match index: {best_template_index} with score {best_match_val:.2f}")
    if best_match_val >= 0.55:
        top_left = best_match_loc
        h, w = best_template_shape
        bottom_right = (top_left[0] + w, top_left[1] + h)

        # Draw match rectangle
        cv2.rectangle(img_rgb, top_left, bottom_right, (0, 255, 0), 2)

        # Center of match
        center_x = top_left[0] + w // 2
        center_y = top_left[1] + h // 2

        # Determine closest corner
        corners = [
            ("top-left", (0, 0)),
            ("top-right", (width, 0)),
            ("bottom-left", (0, height)),
            ("bottom-right", (width, height))
        ]
        
        closest_corner = min(corners, key=lambda c: 
            np.sqrt((center_x - c[1][0])**2 + (center_y - c[1][1])**2))

        # Print image name and closest corner
        if image_name:
            print(f"{image_name}: {closest_corner[0]}")
        else:
            print(f"Closest corner: {closest_corner[0]}")
            
        return closest_corner[0]
    else:
        if image_name:
            print(f"{image_name}: No good match found.")
        else:
            print("No good match found.")
        return None

In [15]:
# Create output directory if it doesn't exist
output_dir = './chessboard_outputs/'
os.makedirs(output_dir, exist_ok=True)

# Process all images in the images folder
# Get all image files in the folder
all_image_files = [os.path.join(BASE_IMG_FOLDER, f) for f in os.listdir(BASE_IMG_FOLDER) 
                  if f.endswith(('.jpg', '.png', '.jpeg'))]

MAX_IMAGES = 50  # Set to None to process all images
image_files = all_image_files[:MAX_IMAGES] if MAX_IMAGES is not None else all_image_files

print(f"Processing {len(image_files)} out of {len(all_image_files)} available images")

# # Define specific image files to process
# image_filenames_failed = [
# "G000_IMG062.jpg",
# "G000_IMG087.jpg",
# ]

# # Create full paths for each image file
# image_files = [os.path.join(BASE_IMG_FOLDER, filename) for filename in image_filenames_failed]

# Process each image
failed_images = []
rotations = {
    "top-left": 90,
    "top-right": 180,
    "bottom-left": 0,
    "bottom-right": -90
}

successful_images = {}

for image_path in image_files:
    filename = os.path.basename(image_path)
    print(f"Processing {filename}")

    if (SHOW_DEBUG_IMGS != "NONE"):
        plt.figure(figsize=(16, 8))

    warped, warped_original_rgb, matrix, ordered_corners, dst_corners = process_chessboard_image(image_path)

    if warped is not None:
        # Detect orientation
        closest_corner = find_horse_template_matching(warped_original_rgb, filename)
        rotation_angle = rotations.get(closest_corner, 0)

        # Apply rotation to both warped images
        def rotate_image(image, angle):
            (h, w) = image.shape[:2]
            center = (w // 2, h // 2)
            M = cv2.getRotationMatrix2D(center, angle, 1.0)
            return cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)

        warped_rotated = rotate_image(warped, rotation_angle)
        warped_original_rgb_rotated = rotate_image(warped_original_rgb, rotation_angle)

        successful_images[filename] = {
            'warped': warped_rotated,
            'warped_original_rgb': warped_original_rgb_rotated,
            'matrix': matrix,
            'ordered_corners': ordered_corners,
            'dst_corners': dst_corners
        }

    else:
        failed_images.append(filename)

    if (SHOW_DEBUG_IMGS != "NONE"):
        plt.tight_layout(pad=3.0)
        subplot_path = os.path.join(output_dir, f"subplots_{filename.split('.')[0]}.png")
        if SAVE_DEBUG_IMGS:
            plt.savefig(subplot_path, dpi=300, bbox_inches='tight')
        print(f"Saved subplots to {subplot_path}")
        plt.show()

# Summary
print("\nImages where chessboard detection failed:")
for failed_image in failed_images:
    print(failed_image)

print(f"\nSuccessfully processed {len(image_files) - len(failed_images)}/{len(image_files)} images")
print(f"Subplot images saved to: {os.path.abspath(output_dir)}")

Processing 50 out of 50 available images
Processing G000_IMG062.jpg
Best match index: 0 with score 0.92
G000_IMG062.jpg: bottom-left
Processing G000_IMG087.jpg
Best match index: 2 with score 0.99
G000_IMG087.jpg: top-right
Processing G000_IMG102.jpg
Best match index: 0 with score 0.76
G000_IMG102.jpg: bottom-left
Processing G006_IMG048.jpg
Best match index: 2 with score 0.96
G006_IMG048.jpg: top-right
Processing G006_IMG086.jpg
Best match index: 2 with score 0.94
G006_IMG086.jpg: top-right
Processing G006_IMG119.jpg
Best match index: 1 with score 0.98
G006_IMG119.jpg: top-left
Processing G019_IMG082.jpg
Best match index: 3 with score 0.96
G019_IMG082.jpg: bottom-right
Processing G028_IMG015.jpg
Best match index: 0 with score 0.87
G028_IMG015.jpg: bottom-left
Processing G028_IMG062.jpg
Best match index: 0 with score 0.92
G028_IMG062.jpg: bottom-left
Processing G028_IMG098.jpg
Best match index: 2 with score 0.91
G028_IMG098.jpg: top-right
Processing G028_IMG101.jpg
Best match index: 0 wi

In [16]:
import matplotlib.pyplot as plt

files = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']

for k, v in successful_images.items():
    board_size = v['warped_original_rgb'].shape[0]
    crop_margin = int(board_size * 0.07)

    cropped_board = v['warped_original_rgb'][crop_margin:board_size - crop_margin,
                                             crop_margin:board_size - crop_margin]
    v['cropped_board'] = cropped_board
    h_lines, v_lines = detect_grid_lines(cropped_board, show_lines=False)

    if len(h_lines) != 7 or len(v_lines) != 7:
        print(f"Not enough grid lines detected in {k}.")
        continue

    height, width = cropped_board.shape[:2]
    h_lines_full = [0] + sorted(h_lines) + [height]
    v_lines_full = [0] + sorted(v_lines) + [width]

    squares = []

    for i in range(8):  # rows (0 = top = rank 8)
        for j in range(8):  # columns (0 = left = file a)
            y1, y2 = h_lines_full[i], h_lines_full[i + 1]
            x1, x2 = v_lines_full[j], v_lines_full[j + 1]
            square = cropped_board[y1:y2, x1:x2]
            rank = 8 - i
            file = files[j]
            label = f"{file}{rank}"
            squares.append({
                "position": (i, j),
                "label": label,
                "image": square
            })

    v['squares'] = squares

### Taking Metrics

In [33]:
files = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
DISTANCE_THRESHOLD = 25
BORDER_TOLERANCE = 5

# Load the XGBoost model
xgb_model = xgb.XGBClassifier()
xgb_model.load_model("xgb_piece_detector.json")

In [34]:
def detect_circles(grey_board, min_radius=20, max_radius=32):
    grey_board = cv2.GaussianBlur(grey_board, (3, 3), 2)
    circles = cv2.HoughCircles(grey_board, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30,
                                param1=60, param2=25, minRadius=min_radius, maxRadius=max_radius)
    if circles is not None:
        return np.round(circles[0, :]).astype("int")
    return []

In [35]:
def euclidean_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

In [36]:
def extract_features(image):
    h, w, _ = image.shape

    # Grayscale version for variance and edge detection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Center region
    center = image[h//4:3*h//4, w//4:3*w//4]
    avg_center_color = np.mean(center)

    # Full square stats
    avg_square_color = np.mean(image)
    color_contrast_center = abs(avg_center_color - avg_square_color)
    color_variance = np.var(image)

    # Top vs bottom
    top_half = image[:h//2, :]
    bottom_half = image[h//2:, :]
    mean_top = np.mean(top_half)
    mean_bottom = np.mean(bottom_half)
    contrast_top_bottom = abs(mean_top - mean_bottom)

    # Edge strength using Sobel
    sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 1, ksize=3)
    edge_strength = np.mean(np.abs(sobel))

    return [
        avg_center_color,
        avg_square_color,
        color_contrast_center,
        color_variance,
        mean_top,
        mean_bottom,
        contrast_top_bottom,
        edge_strength
    ]

In [37]:
def validate_with_model(image):
    # Extract features from the image
    features = np.array(extract_features(image), dtype=np.float32).reshape(1, -1)  # Ensure it's a 2D array (1, num_features)
    
    # Predict using the model directly on the feature array (no need to create DMatrix manually)
    prediction_prob = xgb_model.predict(features)[0]
    print(f"Prediction probability: {prediction_prob}")
    
    # 1 = piece, 0 = empty
    return prediction_prob

In [38]:
def calculate_metrics(detected_matrix, ground_truth_matrix):
    detected_flat = detected_matrix.flatten()
    ground_truth_flat = ground_truth_matrix.flatten()

    tp = np.sum(np.logical_and(detected_flat == 1, ground_truth_flat == 1))
    fp = np.sum(np.logical_and(detected_flat == 1, ground_truth_flat == 0))
    fn = np.sum(np.logical_and(detected_flat == 0, ground_truth_flat == 1))
    tn = np.sum(np.logical_and(detected_flat == 0, ground_truth_flat == 0))

    accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'true_positives': tp,
        'false_positives': fp,
        'false_negatives': fn,
        'true_negatives': tn
    }

In [39]:
def process_image(key, v, real_matrices, show_plots=False):
    board_size = v['warped_original_rgb'].shape[0]
    crop_margin = int(board_size * 0.07)

    cropped_board = v['warped_original_rgb'][crop_margin:board_size - crop_margin,
                                             crop_margin:board_size - crop_margin]
    v['cropped_board'] = cropped_board

    h_lines, v_lines = detect_grid_lines(cropped_board, show_lines=False)

    if len(h_lines) != 7 or len(v_lines) != 7:
        print(f"Not enough grid lines detected in {key}.")
        return None

    height, width = cropped_board.shape[:2]
    h_lines_full = [0] + sorted(h_lines) + [height]
    v_lines_full = [0] + sorted(v_lines) + [width]

    squares = []
    square_centers = []
    square_bounds = []

    for i in range(8):
        for j in range(8):
            y1, y2 = h_lines_full[i], h_lines_full[i + 1]
            x1, x2 = v_lines_full[j], v_lines_full[j + 1]
            square = cropped_board[y1:y2, x1:x2]
            rank = 8 - i
            file = files[j]
            label = f"{file}{rank}"
            squares.append({"position": (i, j), "label": label, "image": square})
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            square_centers.append((center_x, center_y))
            square_bounds.append((x1, y1, x2, y2))

    v['squares'] = squares

    gray = cv2.cvtColor(cropped_board, cv2.COLOR_RGB2GRAY)
    circles = detect_circles(gray)

    filtered_circles = []
    for idx, (cx, cy) in enumerate(square_centers):
        closest_circle = None
        min_dist = float('inf')
        x1, y1, x2, y2 = square_bounds[idx]

        for (x, y, r) in circles:
            dist = euclidean_distance((x, y), (cx, cy))
            if (x - r + BORDER_TOLERANCE >= x1 and x + r - BORDER_TOLERANCE <= x2 and 
                y - r + BORDER_TOLERANCE >= y1 and y + r - BORDER_TOLERANCE <= y2):
                if dist < DISTANCE_THRESHOLD and dist < min_dist:
                    closest_circle = (x, y, r)
                    min_dist = dist

        if closest_circle:
            filtered_circles.append(closest_circle)

    board_matrix = []
    for i in range(len(square_centers)):
        x1, y1, x2, y2 = square_bounds[i]
        presence = 0
        for (x, y, r) in filtered_circles:
            if (x1 <= x <= x2 and y1 <= y <= y2 and 
                x - r + BORDER_TOLERANCE >= x1 and x + r - BORDER_TOLERANCE <= x2 and 
                y - r + BORDER_TOLERANCE >= y1 and y + r - BORDER_TOLERANCE <= y2) and validate_with_model(cropped_board[y1:y2, x1:x2]):
                # Check if the circle is within the square bounds and validate with the model
                presence = 1
                break
        board_matrix.append(presence)

    board_matrix_2d = np.array(board_matrix).reshape(8, 8)
    v['presence_matrix'] = board_matrix_2d

    if key in real_matrices:
        ground_truth_matrix = np.array(real_matrices[key])
        metrics = calculate_metrics(np.flipud(board_matrix_2d), ground_truth_matrix)
        v['detection_metrics'] = metrics

        print(f"Detection metrics for {key}:")
        print(f"  Accuracy: {metrics['accuracy']:.4f}")
        print(f"  Precision: {metrics['precision']:.4f}")
        print(f"  Recall: {metrics['recall']:.4f}")
        print(f"  F1 Score: {metrics['f1_score']:.4f}")
        print(f"  TP: {metrics['true_positives']}, FP: {metrics['false_positives']}, FN: {metrics['false_negatives']}, TN: {metrics['true_negatives']}")

    print(f"Presence Matrix for {key}:")
    print("Number of pieces detected:", np.sum(board_matrix_2d))
    print(np.flipud(board_matrix_2d).tolist())

    # Optional visualization
    if show_plots:
        # Plot with 3 or 4 subplots (4th for ground truth comparison if available)
        num_subplots = 4 if key in real_matrices else 3
        plt.figure(figsize=(18, 6))

        plt.subplot(1, num_subplots, 1)
        plt.imshow(gray, cmap='gray')
        plt.title("Grayscale")
        plt.axis('off')

        # Show circles on the board
        all_circles_image = np.copy(cropped_board)
        if circles is not None:
            for (x, y, r) in circles:
                cv2.circle(all_circles_image, (x, y), r, (0, 0, 255), 2)  # Red circle
                cv2.rectangle(all_circles_image, (x - 3, y - 3), (x + 3, y + 3), (0, 255, 255), -1)  # Yellow center
        
        # Show filtered circles with green color
        for (x, y, r) in filtered_circles:
            cv2.circle(all_circles_image, (x, y), r, (0, 255, 0), 2)  # Green circle for valid circles

        plt.subplot(1, num_subplots, 2)
        plt.imshow(all_circles_image)
        plt.title("Board with Circles (Green = Valid)")
        plt.axis('off')

        plt.subplot(1, num_subplots, 3)
        plt.imshow(board_matrix_2d, cmap='hot', interpolation='nearest')
        plt.title("Detected Presence Matrix")
        plt.axis('off')
            
        plt.tight_layout()
        plt.show()

    return v

In [40]:
presence_matrix = []

for k, v in successful_images.items():
    result = process_image(k, v, REAL_MATRICES, show_plots=False)
    if result:
        presence_matrix.append(result['presence_matrix'])

# Aggregate metrics
all_metrics = [v['detection_metrics'] for v in successful_images.values() if 'detection_metrics' in v]
if all_metrics:
    def average(key): return np.mean([m[key] for m in all_metrics])
    aggregate_metrics = {
        "mean_accuracy": average("accuracy"),
        "mean_precision": average("precision"),
        "mean_recall": average("recall"),
        "mean_f1_score": average("f1_score"),
        "total_true_positives": sum(m["true_positives"] for m in all_metrics),
        "total_false_positives": sum(m["false_positives"] for m in all_metrics),
        "total_false_negatives": sum(m["false_negatives"] for m in all_metrics),
        "total_true_negatives": sum(m["true_negatives"] for m in all_metrics),
    }

    print("\n📊 Aggregate Metrics:")
    for k, v in aggregate_metrics.items():
        print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")
else:
    print("No detection metrics were found.")

Prediction probability: 1
Prediction probability: 1
Prediction probability: 1
Prediction probability: 1
Prediction probability: 1
Prediction probability: 1
Prediction probability: 1
Prediction probability: 1
Prediction probability: 1
Prediction probability: 1
Prediction probability: 1
Prediction probability: 1
Prediction probability: 1
Prediction probability: 1
Prediction probability: 1
Detection metrics for G000_IMG062.jpg:
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1 Score: 1.0000
  TP: 15, FP: 0, FN: 0, TN: 49
Presence Matrix for G000_IMG062.jpg:
Number of pieces detected: 15
[[1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 1], [0, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 1, 1, 0, 0, 0], [0, 1, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 1, 1], [0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0]]
Prediction probability: 1
Prediction probability: 1
Prediction probability: 1
Prediction probability: 1
Prediction probability: 1
Prediction probability: 1
Prediction probability: 1
Predict